In [5]:
!pip install catboost sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 6.9 MB/s eta 0:00:00


# Чтение, подготовка, обработка данных

In [2]:
import pandas as pd

df = pd.read_csv('augmented_hardware_type.csv')

In [3]:
df['Описание'] = df['Описание'].str.replace('_x000D_', '')

Извлечение признаков из текста

In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')

def embed_text(texts, model):
    embeddings = model.encode(texts, show_progress_bar=True, batch_size=32, device='cuda')
    return embeddings

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

In [7]:
from tqdm import tqdm

tqdm.pandas()

tqdm.pandas(desc="Эмбеддинг текстов")
X_theme = embed_text(df['Тема'].values, model).tolist()
X_desc = embed_text(df['Описание'].values, model).tolist()

df['Тема_emb'] = X_theme
df['Описание_emb'] = X_desc

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Создание выборок для обучения и теста

In [8]:
import numpy as np

X = np.hstack([X_theme, X_desc])

In [9]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Тип оборудования'])

class_labels = label_encoder.classes_

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
import gc
import torch

torch.cuda.empty_cache()
gc.collect()

# Обучение модели классификации типа оборудования на данных

In [23]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.005,
    depth=10,
    random_seed=42,
    auto_class_weights='Balanced',
    verbose=200,
    task_type="GPU",
    devices='0'
)

In [24]:
model.fit(X_train, y_train)

0:	learn: 1.0907502	total: 450ms	remaining: 7m 29s
200:	learn: 0.3925287	total: 51.8s	remaining: 3m 25s
400:	learn: 0.1958816	total: 1m 43s	remaining: 2m 35s
600:	learn: 0.1183947	total: 2m 36s	remaining: 1m 44s
800:	learn: 0.0826298	total: 3m 30s	remaining: 52.3s
999:	learn: 0.0632220	total: 4m 24s	remaining: 0us


Замеры метрик

In [27]:
y_pred = model.predict(X_test)

In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.9424
Precision: 0.9466
Recall: 0.9424
F1 Score: 0.9420


Сохранение модели

In [29]:
model.save_model('hardware_type_classifier.cbm', format='cbm')